#### 상대모멘텀
- 10개의 종목을 이용하여 투자 전략 시스템

    1. 월초부터 월말까지의 수정종가를 이용하여 수익율 생성
    2. 월별 수익율이 높은 n개를 선택
    3. 해당하는 종목을 매수하여 해당하는 종목에 신호가 없어지면 매도하는 형식으로 수익율 생성

In [2]:
import pandas as pd
import numpy as np
import os
from glob import glob
import warnings

In [3]:
# warning 메시지 출력 방지
warnings.filterwarnings('ignore')

In [8]:
# 웗별 수익율을 계산하는 함수
def create_1m_rtn(_df,
                  _ticker,
                  _start = '2010-01-01',
                  _col = 'Adj Close'):
    # 복사본 생성
    result = _df.copy()
    # Date 컬럼이 존재하면 index로 변경
    if 'Date' in result.columns:
        result.set_index('Date', inplace=True)
    # index를 시계열로 변경
    result.index = pd.to_datetime(result.index)
    # 시작 시간과 기준이 되는 컬럼으로 필터링
    result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1n-rtn'] = 0
    result['CODE'] = _ticker
    # 기준년월(STD-YM)의 유니크한 리스트를 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [9]:
aapl = pd.read_csv("../../csv/AAPL.csv")

In [10]:
sample_aapl, ym_list = create_1m_rtn(aapl, 'AAPL')

In [11]:
sample_aapl

,Adj Close,STD-YM,1n-rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL
...,...,...,...,...
2019-06-18,198.449997,2019-06,0,AAPL
2019-06-19,197.869995,2019-06,0,AAPL
2019-06-20,199.460007,2019-06,0,AAPL


In [12]:
ym_list

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', 

In [13]:
# listdir() 파일의 이름의 목록
os.listdir('./data')
# glob() 파일의 경로와 이름의 목록
files = glob('./data/*.csv')